# Compute energy levels of Hydrogen Atom with Gaussian Linear Combination of Atomic Orbitals 


Here we will calculate a linear combination of atomic orbitals (LCAO) for the Hydrogen atom. Slater orbitals are exact in this case, so let's try a Gaussian orbital to demonstrate the procedure. 


### Import path and add our software

In [1]:
import os
import sys
import scipy.optimize

### Add the cpt BFGS minimization, initialize constants for 2 Gaussians

In [2]:
import numpy as np

# physical constants
hbar = 1.0                  # Planck's constant / 2pi
m = 1.0                     # electron mass
e = 1.0                     # proton charge

# LCAO variational wave function
# psi = sum( d_i g(alpha_i, r) ) for i = 0, 1, 2, ...
# assume d_0 = 1 and vary alpha_0, d_1, alpha_1, d_2, alpha_2, ...
# vector of variational parameters
p = np.array([ 1.0, 1.0, 1.0, 0.5 ])       # initial guess for [ d_0, alpha_0, d_1, alpha_1 ]
accuracy = 1.0e-6           # desired accuracy for numerical operations

### Define the g function, matrix elements, minimization function and derivative

In [3]:
class LCAOGauss:
    def __init__(self, p):
        self.d = p[0::2]
        self.alpha = p[1::2]        
        self.ii = np.arange(len(self.alpha))
        self.jj = np.arange(len(self.alpha))
        self.i, self.j = np.meshgrid(self.ii,self.jj)        

    def Sij(self):  # matrix elements of S
        return (np.pi / (self.alpha[self.i] + self.alpha[self.j]))**(3.0/2.0)

    def Tij(self):  # matrix elements of T
        return (3.0 * hbar**2 / m * self.alpha[self.i] * self.alpha[self.j] *
                np.pi**(3.0/2.0) / (self.alpha[self.i] + self.alpha[self.j])**(5.0/2.0))
    
    def Vij(self):  # matrix elements of V
        return - 2.0 * e**2 * np.pi / (self.alpha[self.i] + self.alpha[self.j])

    def E(self):            # energy as function of N alpha_i and d_i
        S = H = 0.0        
        fac = (self.alpha[self.i] * self.alpha[self.j])**(3.0/4.0)* self.d[self.i] * self.d[self.j]
        Hvals = fac * (self.Tij() + self.Vij() )
        Svals = fac * self.Sij()
        H = np.sum(Hvals)
        S = np.sum(Svals)
        return H / S

    def __call__(self, p):                # function for BFGS minimization
        # assume p = [ d0, alpha_0, d_1, alpha_1, d_2, alpha_2, ... ]
        self.alpha = np.where( p[1::2] > accuracy, p[1::2], accuracy)
        self.d = p[::2]    
        e = self.E()
        return e

    def norm(self):                # norm of LCAO
        Sijvals = self.Sij() * self.d[self.i] * self.d[self.j]
        return np.linalg.norm(Sijvals)


### Drive the simulation

In [4]:

print(" Variational method for Hydrogen using Gaussian LCAO")
print(" Minimize <psi|H|psi>/<psi|psi> using BFGS algorithm")
lcao = LCAOGauss(p)
res = scipy.optimize.minimize(lcao, p, tol=accuracy)
alpha = res.x[1::2]
d = res.x[::2]
A = lcao.norm()
d /= A

print( "%3s %7s %7s" % ( 'i', 'alpha', 'd') )
for i in range(len(alpha)):
    print ( "%3d %6.5f %6.5f" % (i, alpha[i], d[i]) )

 Variational method for Hydrogen using Gaussian LCAO
 Minimize <psi|H|psi>/<psi|psi> using BFGS algorithm
  i   alpha       d
  0 1.33250 0.01072
  1 0.20153 0.03208
